In [1]:
import torch
import json
import pandas as pd
import numpy as np

torch.__version__

'1.12.1+cu113'

In [2]:
torch.cuda.is_available()

True

In [5]:
FILE = '../../arxiv-metadata-oai-snapshot.json'

def get_data():
    with open(FILE) as f:
        for line in f:
            yield line

### TODO: dataframe에 JSON LOAD Columns 모두 추가해서 csv 저장

In [6]:
dataframe = {
    'id': [],
    'title': [],
    'year': [],
    'abstract': [],
    'author': []
}

data = get_data()
for i, paper in enumerate(data):
    paper = json.loads(paper)
    # print(paper)
    # break
    try:
        date = int(paper['update_date'].split('-')[0])
        if date > 2019:
            dataframe['id'].append(paper['id'])
            dataframe['title'].append(paper['title'])
            dataframe['year'].append(date)
            dataframe['abstract'].append(paper['abstract'])
            dataframe['author'].append(paper['submitter'])
    except: pass

데이터 너무 많음 -> cs 관련?
인용수 -> 적은 애들 neg sample로 사용 가능

graph -> 인접행렬

In [8]:
df = pd.DataFrame(dataframe)

In [9]:
del dataframe

In [10]:
df

,id,title,year,abstract,author
0,0704.0033,Convergence of the discrete dipole approximati...,2022,We performed a rigorous theoretical converge...,Maxim A. Yurkin
1,0704.0038,The discrete dipole approximation: an overview...,2022,We present a review of the discrete dipole a...,Maxim A. Yurkin
2,0704.0275,Mapping radii of metric spaces,2021,It is known that every closed curve of lengt...,George M. Bergman
3,0704.0348,What can emission lines tell us?,2023,1 Generalities\n 2 Empirical diagnostics ba...,Grazyna Stasinska
4,0704.0479,The affine part of the Picard scheme,2021,We describe the maximal torus and maximal un...,Thomas Geisser H
...,...,...,...,...,...
886395,solv-int/9502003,The Hamiltonian structure of the dispersionles...,2020,The Hamiltonian structure of the two-dimensi...,David Fairlie
886396,solv-int/9807001,DNA Transcription Mechanism with a Moving Enzyme,2021,Previous numerical investigations of an one-...,Juhi-Lian Julian Ting
886397,solv-int/9807004,Degenerate Frobenius manifolds and the bi-Hami...,2020,The bi-Hamiltonian structure of certain mult...,I. A. B. Strachan
886398,solv-int/9808019,Hypercomplex Integrable Systems,2020,In this paper we study hypercomplex manifold...,James D. E. Grant


In [16]:
df.loc[df['title'] == 'Rethinking Attention with Performers']

,id,title,year,abstract,author
228644,2009.14794,Rethinking Attention with Performers,2022,"We introduce Performers, Transformer archite...",Valerii Likhosherstov


attention is all you need 16667
CCNet: Criss-Cross Attention for Semantic Segmentation 36734
Heterogeneous Graph Attention Network 45683
DeBERTa: Decoding-enhanced BERT with Disentangled Attention 170515
Rethinking Attention with Performers 228644

[16667, 36734, 45683, 170515, 228644]



In [9]:
max([len(i) for i in df.abstract])

5089

In [10]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

loading existing colbert_linear and sparse_linear---------


In [11]:
# 두 시간 걸리니 참고만 하셈
output = model.encode(list(df.abstract),
                      batch_size=12, 
                      max_length=5090, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                      )['dense_vecs']

Inference Embeddings: 100%|██████████| 73867/73867 [1:57:03<00:00, 10.52it/s]  


In [12]:
emb_df = pd.DataFrame(output)

In [13]:
emb_df.to_csv('embedding.csv', index=False)

# Inference

In [18]:
output = pd.read_csv('../../embedding.csv').to_numpy()

In [19]:
from sklearn.neighbors import KNeighborsClassifier

In [20]:
y = df.index

In [21]:
nn = KNeighborsClassifier(n_neighbors=6)
nn.fit(output, y)

KNeighborsClassifier(n_neighbors=6)

In [12]:
import random

맨처음 시나리오 있으면 좋겠다.

In [40]:
data = {i:[] for i in df.columns}
for idx in [16667, 36734, 45683, 170515, 228644]:
    sample = df["title"][idx]
    for i in df.columns: data[i].append(df[i][idx])
    dist, index = nn.kneighbors(X=output[idx,:].reshape(1,-1))
    print(f"Sample:\n{sample}\n")
    # for i in range(1,6):
    #     print(f"Recommendation {i}:\n{df['title'][index[0][i]]}\n")
    #     for j in df.columns: data[j].append(df[j][index[0][i]])
    for k in range(1,6):
        print(f"Recommendation {i}-{k-1}:\n{df['title'][index[0][k]]}\n")
        for j in df.columns: data[j].append(df[j][index[0][k]])

    print("===============\n")

Sample:
Attention Is All You Need

Recommendation author-0:
Fixed Encoder Self-Attention Patterns in Transformer-Based Machine
  Translation

Recommendation author-1:
Transformer++

Recommendation author-2:
An Efficient Transformer Decoder with Compressed Sub-layers

Recommendation author-3:
Context- and Sequence-Aware Convolutional Recurrent Encoder for Neural
  Machine Translation

Recommendation author-4:
A Cost-Efficient FPGA Implementation of Tiny Transformer Model using
  Neural ODE


Sample:
CCNet: Criss-Cross Attention for Semantic Segmentation

Recommendation author-0:
CTNet: Context-based Tandem Network for Semantic Segmentation

Recommendation author-1:
TriangleNet: Edge Prior Augmented Network for Semantic Segmentation
  through Cross-Task Consistency

Recommendation author-2:
Real-time Semantic Segmentation via Spatial-detail Guided Context
  Propagation

Recommendation author-3:
BiCANet: Bi-directional Contextual Aggregating Network for Image
  Semantic Segmentation

Reco

In [41]:
len(data['id'])

30

In [25]:
import random

In [96]:
graph = {
    'nodes': [
            {
                'title': data['title'][i], 
                'author': data['author'][i],
                'year': int(data['year'][i]),
                'id': data['id'][i],
                'height': 2 if i==0 else 1,
                'size': random.randint(10,20), # citation num
                'color': f'rgb(240, 237, 207, {(data["year"][i] - 2015) / 20})' # publish year
            }
        for i in range(30)], 
    'links': [
            {
                'source': data['id'][i],
                'target': data['id'][j],
                'distance': int(random.randrange(1, 100)) # 100 * similarity
            }
        for i in range(0, 30, 6) for j in range(i+1, i+6)]
    }

In [97]:
iter = 0
for i in graph['nodes']:
    if iter % 6 == 0:
        i['size'] = 80
        i['color'] = f'rgb(11, 96, 176, 0.8)'
    iter += 1
        

In [78]:
data

{'id': ['1706.03762',
  '2002.10260',
  '2003.04974',
  '2101.00542',
  '2101.04030',
  '2401.02721',
  '1811.11721',
  '2104.09805',
  '2210.05152',
  '2005.11034',
  '2003.09669',
  '2010.04962',
  '1903.07293',
  '2112.06244',
  '2103.00118',
  '2207.02547',
  '2007.08294',
  '2207.11761',
  '2006.03654',
  '2401.15861',
  '2204.08688',
  '2104.11559',
  '2306.01497',
  '2003.02912',
  '2009.14794',
  '2006.03555',
  '2202.08791',
  '2103.02143',
  '2310.11685',
  '2209.10876'],
 'title': ['Attention Is All You Need',
  'Fixed Encoder Self-Attention Patterns in Transformer-Based Machine\n  Translation',
  'Transformer++',
  'An Efficient Transformer Decoder with Compressed Sub-layers',
  'Context- and Sequence-Aware Convolutional Recurrent Encoder for Neural\n  Machine Translation',
  'A Cost-Efficient FPGA Implementation of Tiny Transformer Model using\n  Neural ODE',
  'CCNet: Criss-Cross Attention for Semantic Segmentation',
  'CTNet: Context-based Tandem Network for Semantic Seg

In [ ]:
r"""
nodes: [
    {                   # paper에 대한 정보
        id: str -> xxxx.xxxx           # 챗봇 연결링크 생성시 필요, Nullable: False
        title: str -> Attention is ~   # 유저한테 보여줄 논문 제목 -> 만드는중
        size: int                      # 인용수
        hieght: int                    # 부모 노드에 연결될 Edge 굵기: 1hop 2hop 구분으로 사용?
        color: int                     # 노드 특성 반영한 한가지 값. 투명도로 조절해서 표시할 수 있음.
    }, ...
],
edges: [
    {
        source: str -> id           # 자식 노드 제목
        to: str -> id               # 부모 노드 제목
        distance -> int [0 ~ 1]     # 모델 output 유사도
    }, ...
]
"""

In [86]:
graph

{'nodes': [{'title': 'Attention Is All You Need',
   'author': 'Llion Jones',
   'year': 2023,
   'id': '1706.03762',
   'height': 2,
   'size': 80,
   'color': 'rgb(240, 237, 207, 0.8)'},
  {'title': 'Fixed Encoder Self-Attention Patterns in Transformer-Based Machine\n  Translation',
   'author': 'Alessandro Raganato',
   'year': 2020,
   'id': '2002.10260',
   'height': 1,
   'size': 77,
   'color': 'rgb(11, 96, 176, 0.25)'},
  {'title': 'Transformer++',
   'author': 'Prakhar Thapak',
   'year': 2020,
   'id': '2003.04974',
   'height': 1,
   'size': 34,
   'color': 'rgb(11, 96, 176, 0.25)'},
  {'title': 'An Efficient Transformer Decoder with Compressed Sub-layers',
   'author': 'Ye Lin',
   'year': 2023,
   'id': '2101.00542',
   'height': 1,
   'size': 43,
   'color': 'rgb(11, 96, 176, 0.4)'},
  {'title': 'Context- and Sequence-Aware Convolutional Recurrent Encoder for Neural\n  Machine Translation',
   'author': 'Seba Susan',
   'year': 2021,
   'id': '2101.04030',
   'height': 1,

In [98]:
import os
import json
with open(os.path.join('back/data', 'attentiongraph.json'), 'w') as json_file:
    json.dump(graph, json_file, ensure_ascii=False) # ensure_ascii True 하면 한글이 안보임 ㅋㅋ